 # さまざまな予測性能評価

In [26]:
import unittest
import doctest
import os
path = os.path.dirname(os.path.abspath(__file__))

 ## 回帰の予測性能評価

 ### 平均絶対誤差（MAE）の復習

 ### 平均二乗誤差（MSE）の復習

 #### コード13-1 データとモデルの準備

In [27]:
import pandas as pd
# 欠損値があるままでは学習できないので欠損値処理だけ行う
df = pd.read_csv(path + '/data/cinema.csv')
df = df.fillna(df.mean())
x = df.loc[:, 'SNS1':'original']
t = df['sales']
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, t)

LinearRegression()

 #### コード13-2 平均2乗誤差を計算する

In [28]:
from sklearn.metrics import mean_squared_error
# 訓練データでのMSE値
pred = model.predict(x)

mse = mean_squared_error(t, pred)
mse

151986.03957624515

 ### ２乗平均平方根誤差（RMSE）

 $$
 RMSE = \sqrt{MSE}
 $$

 #### コード13-3 RMSEの計算

In [29]:
import math
math.sqrt(mse) # RMSEの計算

389.853869515547

 #### コード13-4 予測結果と実際の誤差を検証する

In [30]:
from sklearn.metrics import mean_absolute_error
yosoku = [2, 3, 5, 7, 11, 13] # 予測結果をリストで作成
target = [3, 5, 8, 11, 16, 19] # 実際の結果をリストで作成

mse = mean_squared_error(target, yosoku)
print('rmse:{}'.format(math.sqrt(mse)))
print('mae:{}'.format(mean_absolute_error(target, yosoku)))

print('外れ値の混入')
yosoku = [2, 3, 5, 7, 11, 13, 46] # 実際には23だけど46と予測
target = [3, 5, 8, 11, 16, 19, 23]
mse = mean_squared_error(target, yosoku)
print('rmse:{}'.format(math.sqrt(mse)))
print('mae:{}'.format(mean_absolute_error(target, yosoku)))

rmse:3.8944404818493075
mae:3.5
外れ値の混入
rmse:9.411239481143202
mae:6.285714285714286


 ## 分類の予測性能評価

 ### 適合率と再現率

 #### コード13-5 データの準備

In [31]:
# データの準備
df = pd.read_csv(path + '/data/Survived.csv')
df = df.fillna(df.mean())

x = df[['Pclass', 'Age']]
t = df['Survived']

 #### コード13-6 モデルの準備

In [32]:
# モデルの準備
from sklearn import tree
model = tree.DecisionTreeClassifier(max_depth=2, random_state=0)
model.fit(x, t)

DecisionTreeClassifier(max_depth=2, random_state=0)

 #### コード13-7 再現率と適合率を一括で計算

In [33]:
from sklearn.metrics import classification_report
pred = model.predict(x)
out_put = classification_report(t, pred)
print(out_put)

              precision    recall  f1-score   support

           0       0.78      0.65      0.71       549
           1       0.56      0.70      0.62       342

    accuracy                           0.67       891
   macro avg       0.67      0.68      0.67       891
weighted avg       0.69      0.67      0.68       891



 #### コード13-8 classification_report関数にパラメータ引数を指定

In [34]:
out_put = classification_report(t, pred, output_dict=True)

# out_putをデータフレームに変換
pd.DataFrame(out_put)

,0,1,accuracy,macro avg,weighted avg
precision,0.778742,0.558140,0.672278,0.668441,0.694066
recall,0.653916,0.701754,0.672278,0.677835,0.672278
f1-score,0.710891,0.621762,0.672278,0.666326,0.676680
support,549.000000,342.000000,0.672278,891.000000,891.000000


 ### f1-score

 ## K分割交差検証

 ### ホールドアウト法の問題点

 ### K分割交差検証

 #### コード13-9 K分割交差検証のためのデータ準備

In [35]:
df = pd.read_csv(path + '/data/cinema.csv')
# 学習できないので欠損値処理だけ行う
df = df.fillna(df.mean())
x = df.loc[:, 'SNS1':'original']
t = df['sales']

 #### コード13-10 KFoldの処理で分割時の条件を指定

In [36]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True, random_state=0)

 #### コード13-11 cross_validate関数で交差検証を行う

In [37]:
from sklearn.model_selection import cross_validate
model = LinearRegression()
result = cross_validate(model, x, t, cv=kf, scoring='r2', return_train_score=True)
print(result)

{'fit_time': array([0.00399828, 0.00200033, 0.00301695]), 'score_time': array([0.00199962, 0.00199866, 0.00097942]), 'test_score': array([0.72465051, 0.71740834, 0.75975591]), 'train_score': array([0.76928501, 0.76368104, 0.75780074])}


 #### コード13-12 平均値を計算する

In [38]:
sum(result['test_score']) / len(result['test_score'])

0.7339382541774341

 ### 分類モデルを作るときの交差検証の注意点

 #### コード13-13 StratifiedKFoldのインポート

In [39]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

 ### チューニング後の処理

In [40]:
doctest.testmod(verbose=True)
unittest.main(argv=[''], verbosity=2, exit=False)

3 items had no tests:
    __main__
    __main__.__VSCODE_compute_hash
    __main__.__VSCODE_wrap_run_cell
0 tests in 3 items.
0 passed and 0 failed.
Test passed.



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK
